In [ ]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import io
import glob
import re
import nltk
import matplotlib.pyplot as plt
import pymorphy2
import fitz
import string
import re
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.tokenize import word_tokenize
from tqdm.auto import tqdm, trange

Изначальные признаки: 

__TitleCompany__ - Название компании

__Description__ - Описание компании

__Reiting__ - Рейтинг компании

__Categories__ - Категории компании

__Date__ - Дата опубликования статьи (в будущем удален, как ненужный признак)

__TextPost__ - необработанный текст (в будущем удален, как ненужный признак)

__TitleCompany__ - Название компании

__Description__ - Описание компании

__Reiting__ - Рейтинг компании

__Categories__ - Категории компании

__TextPostLemat__ - обработанный, лематизированный текст 

__TextPostToken__ - обработанный, токенизированный и лематизированный текст (лематизирован другим способом нежели TextPostLemat)

In [ ]:
pip install pymystem3

In [ ]:
pip install fitz

In [ ]:
pip install PyMuPDF

In [ ]:
#выводим общую информацию о пдф файле
pdf_document = r"C:\Users\essww\OneDrive\Рабочий стол\Компании.pdf"
doc = fitz.open(pdf_document)
print("Исходный документ: ", doc)
print("\nКоличество страниц: %i\n\n------------------\n\n" % doc.page_count)
print(doc.metadata)
text_pages = ""
for current_page in range(len(doc)):
    page = doc.load_page(current_page)
    page_text = page.get_text("text")
    # записываем текст пдф файла
    text_pages += page.get_text("text")
    print("Стр. ", current_page+1, "\n")
    print(page_text
# Разделяем текст на строки
text_pages = text_pages.split("\n")
text_new_pages = text_pages[0:26] + text_pages[39:132]
for i in tqdm(range(0,len(text_new_pages),6)):
    TitleCompany.append(text_new_pages[i])
    Description.append(text_new_pages[i+1])
    Reiting.append(text_new_pages[i+4])
    Categories.append(text_new_pages[i+3].replace('�', ' '))

In [ ]:
pagenum = 1
for i in tqdm(range(25)):
    url = "https://habr.com/ru/all/page" + str(pagenum) + '/'
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    pages = soup.find_all('h2', class_='tm-title tm-title_h2')
    for i in pages:
        if 'companies' in str(i.a.get('href')):
            url = 'https://habr.com' + str(i.a.get('href'))
            page = requests.get(url)
            soup = bs(page.text, 'html.parser')
            TitleCompany = soup.find('a', class_='tm-company-snippet__title')
            for i in range(20):
                if TitleCompany.text in df['TitleCompany'][i]:
                    dfHabr["TitleCompany"].append(df['TitleCompany'][i])
                    dfHabr["Description"].append(df['Description'][i])
                    dfHabr["Reiting"].append(df['Reiting'][i])
                    dfHabr["Categories"].append(df['Categories'][i])
                    TextPost = soup.find('div', class_='tm-article-body')
                    TextPost.text.replace('\n', '')
                    TextPost.text.replace('\r', '')
                    dfHabr["TextPost"].append(TextPost.text)
    pagenum = pagenum + 1

In [ ]:
# метод удаления пунктуации
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

# метод удаления чисел
def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

# метод удаления двойных пробелов
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

# метод удаления оставшихся чисел
def remove_notalpha(text):
    return ''.join([i if i.isalpha() else ' ' for i in text])

def remove_english_word(text):
    return ''.join(re.sub(r"[^а-яА-Я\s]+", "",text))

# метод удаления оставшихся символов
st = '❯\xa0'
def remove_othersymbol(text):
    return ''.join([ch if ch not in st else ' ' for ch in text])

# метод токенизирования
def tokenize(text):
    t = word_tokenize(text)
    return [token for token in t if token not in russian_stopwords]

def remove_small_text(text):
    words = text.split()
    return ' '.join([i if len(i) >=3 else ' ' for i in words])

mystem = Mystem() 
# добавление стопслов
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'быть', 'r', 'n', 'а', 'мы', 'с', 'для', 'ещё', 'его', 'также', 'к', 'тем', 'кто', 'чтобы', 'но', 'они', 'будут', 'так', 'где', 'один', 'он ', 'и', 'на', 'но', 'или', 'либо', 'это', 'мб', 'далее', 'дв', 'свой', 'ваш','всё', 'очень', 'её', 'ещё', 'вообще', 'наш', 'который'])
# обработка текста с лематизацией для токенизации
prep_text = [tokenize(remove_multiple_spaces(remove_small_text(remove_notalpha(remove_othersymbol(remove_english_word(remove_numbers(remove_punctuation(text.lower())))))))) for text in tqdm(dfHabr1['TextPost']) if text not in russian_stopwords]
dfHabr1['TextPostToken'] = prep_text
dfHabr1.drop(columns=['TextPost'], axis= 1 , inplace= True )
lemm_texts_list = []
for text in tqdm(dfHabr1['TextPostLemat']):
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
        
    
dfHabr1['TextPostLemat'] = lemm_texts_list

In [ ]:
# сохраняем DataFrame в файл
dfHabr1.to_csv('Report1.csv', index=False)